In [2]:
import censusdata
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')
import geopandas as gpd

In [3]:
#censusdata.search('acs5', 2015, 'concept', 'age')

In [4]:
#censusdata.printtable(censusdata.censustable('acs5', 2015, 'B01001'))

In [78]:
df = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B01001_001E', 'B01001_002E', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E',
                              'B01001_011E','B01001_012E', 'B01001_013E', 'B01001_014E', 'B01001_015E', 'B01001_016E', 
                              'B01001_017E','B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_026E', 
                              'B01001_027E','B01001_028E', 'B01001_029E', 'B01001_030E', 'B01001_031E'])
df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B01001_001E' : 'Total', 'B01001_002E' : 'Male', 'B01001_007E' : 'M 18-19', 'B01001_008E' : 'M 20-24', 
                             'B01001_009E' : 'M 25-29', 'B01001_010E' : 'M 30-34','B01001_011E' : 'M 35-44','B01001_012E' : 'M 45-54', 
                             'B01001_013E' : 'M 55-64', 'B01001_014E' : 'M 65-74', 'B01001_015E' : 'M 75-84', 
                             'B01001_016E' : 'M 85+', 'B01001_017E' : 'Female', 'B01001_022E' : 'F 18-19', 'B01001_023E' : 'F 20-24', 
                             'B01001_024E' : 'F 25-29', 'B01001_025E' : 'F 30-34', 'B01001_026E' : 'F 35-44', 'B01001_027E' : 'F 45-54',
                             'B01001_028E' : 'F 55-64', 'B01001_029E' : 'F 65-74', 'B01001_030E' : 'F 75-84', 'B01001_031E' : 'F 85+'}, 
                              inplace = True)
for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

young_pop = df['M 18-19'] + df['M 20-24'] + df['M 25-29'] + df['M 30-34'] + df['M 35-44'] + \
            df['F 18-19'] + df['F 20-24'] + df['F 25-29'] + df['F 30-34'] + df['F 35-44']

older = df['M 45-54'] + df['M 55-64'] + df['M 65-74'] + df['M 75-84'] + df['M 85+'] + \
        df['F 45-54'] + df['F 55-64'] + df['F 65-74'] + df['F 75-84'] + df['F 85+']

df['18-44'] = young_pop

df['44+'] = older

df = df[['TRACT', 'Total', '18-44', '44+']]

df

,TRACT,Total,18-44,44+
0,802,6701,5189,1575
1,10600,3168,2243,722
2,40100,2442,1584,590
3,40300,4665,3031,1642
4,50400,2284,1263,842
...,...,...,...,...
199,82100,5998,4241,1353
200,60200,2103,1464,538
201,90400,4396,2999,1165
202,120500,2850,1896,741


In [79]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

prct_age = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df.columns[1:])

prct_age.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_age)):
    prct = prct_age['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_age[c].iloc[i] += round(df[filt][c] * dec)

In [80]:
for i in range(len(prct_age)):
    if prct_age.loc[i, 'WARD_PRECI'] == '0502A':
        print(i)
        
prct_age.loc[250, 'WARD_PRECI'] = 502

250


In [81]:
for i in range(len(prct_age)):
    prct_age.loc[i, '18-44'] = round(prct_age.loc[i, '18-44']/prct_age.loc[i, 'Total'],3)
    prct_age.loc[i, '44+'] = round(prct_age.loc[i, '44+']/prct_age.loc[i, 'Total'],3)

prct_age = prct_age.astype({"WARD_PRECI": int})
    
prct_age = prct_age.sort_values(by=['WARD_PRECI']).reset_index(drop=False)

prct_age = prct_age.drop('index', axis=1)

#prct_age = prct_age.drop('level_0', axis=1)


display(prct_age)
prct_age.to_csv('Precinct_Young_Old_Population_Breakdown_2015-2019.csv')

,WARD_PRECI,Total,18-44,44+
0,101,1194.0,0.647,0.302
1,102,2633.0,0.648,0.360
2,103,3499.0,0.588,0.355
3,104,2319.0,0.610,0.372
4,105,4018.0,0.527,0.398
...,...,...,...,...
249,2209,1521.0,0.689,0.289
250,2210,2604.0,0.724,0.268
251,2211,2548.0,0.751,0.242
252,2212,2984.0,0.724,0.246


In [41]:
'0113' in prct_age['WARD_PRECI'].unique()

False

In [272]:
prct_age.to_csv('Precinct_Age_Distribution_Based_On_2011-15_ACS_Census_Data.csv')

In [273]:
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')

props = pd.read_csv('Neighborhood_Make_Up_Of_Precincts.csv')

neigh_age_11_15 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_age.columns[1:])
neigh_age_11_15.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_age_11_15)):
    neighb = neigh_age_11_15['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_age['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_age.columns[1:]:
                try:
                    neigh_age_11_15[c].iloc[i] += round(prct_age[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_age_11_15.columns:
    if c != 'NEIGHBORHOOD':
        neigh_age_11_15[c] = neigh_age_11_15[c].round().astype(int)
        
sum_column = neigh_age_11_15['F 55-64'] + neigh_age_11_15['F 65-74'] + neigh_age_11_15['F 75-84'] + neigh_age_11_15['F 85+']
neigh_age_11_15.insert(23, 'F 55+', sum_column)

sum_column = neigh_age_11_15['M 55-64'] + neigh_age_11_15['M 65-74'] + neigh_age_11_15['M 75-84'] + neigh_age_11_15['M 85+']
neigh_age_11_15.insert(12, 'M 55+', sum_column)

sum_column = neigh_age_11_15['F 55+'] + neigh_age_11_15['M 55+']
neigh_age_11_15['Total 55+'] = sum_column

sum_column = neigh_age_11_15['F 18-19'] + neigh_age_11_15['F 20-24'] + neigh_age_11_15['F 25-29'] + \
             neigh_age_11_15['M 18-19'] + neigh_age_11_15['M 20-24'] + neigh_age_11_15['M 25-29']
neigh_age_11_15['Total 18-29'] = sum_column


cols = list(neigh_age_11_15.columns)[2:]

for c in cols:
    for i in range(len(neigh_age_11_15)):
        neigh_age_11_15.loc[i, c] = round(neigh_age_11_15.loc[i, c]/neigh_age_11_15.loc[i, 'Total'], 3)

neigh_age_11_15

,NEIGHBORHOOD,Total,Male,M 18-19,M 20-24,M 25-29,M 30-34,M 35-44,M 45-54,M 55-64,...,F 30-34,F 35-44,F 45-54,F 55-64,F 65-74,F 75-84,F 55+,F 85+,Total 55+,Total 18-29
0,Roslindale,29018,0.461,0.008,0.006,0.008,0.012,0.039,0.039,0.037,...,0.005,0.539,0.032,0.027,0.022,0.020,0.079,0.009,0.220,0.045
1,Jamaica Plain,42757,0.466,0.009,0.009,0.011,0.024,0.066,0.051,0.038,...,0.004,0.534,0.026,0.021,0.018,0.019,0.069,0.011,0.195,0.051
2,Mission Hill,13921,0.509,0.045,0.036,0.035,0.093,0.071,0.026,0.027,...,0.004,0.490,0.015,0.018,0.013,0.011,0.090,0.048,0.164,0.139
3,Longwood,6240,0.387,0.100,0.048,0.034,0.056,0.037,0.012,0.014,...,0.002,0.613,0.005,0.020,0.007,0.007,0.227,0.193,0.270,0.191
4,Bay Village,1398,0.476,0.021,0.008,0.006,0.014,0.054,0.069,0.024,...,0.019,0.525,0.016,0.014,0.009,0.006,0.074,0.045,0.172,0.075
5,Leather District,537,0.467,0.048,0.020,0.013,0.020,0.054,0.035,0.026,...,0.009,0.533,0.019,0.009,0.004,0.007,0.115,0.095,0.214,0.112
6,Chinatown,2718,0.468,0.046,0.019,0.013,0.019,0.053,0.040,0.025,...,0.011,0.532,0.018,0.010,0.005,0.008,0.113,0.090,0.209,0.109
7,North End,6962,0.516,0.003,0.005,0.009,0.048,0.146,0.078,0.039,...,0.008,0.484,0.010,0.006,0.005,0.000,0.017,0.005,0.131,0.056
8,Roxbury,50606,0.474,0.029,0.017,0.016,0.035,0.044,0.027,0.023,...,0.003,0.526,0.028,0.030,0.026,0.023,0.107,0.029,0.210,0.083
9,South End,23662,0.514,0.010,0.008,0.011,0.029,0.067,0.066,0.032,...,0.010,0.486,0.025,0.014,0.017,0.006,0.054,0.016,0.194,0.057


In [143]:
neigh_age_11_15.to_csv('Boston_Neighborhood_Estimated_Age_Distribution_ACS_2011-2015.csv')

In [152]:
df2 = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B01001_001E', 'B01001_002E', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E',
                              'B01001_011E','B01001_012E', 'B01001_013E', 'B01001_014E', 'B01001_015E', 'B01001_016E', 
                              'B01001_017E','B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_026E', 
                              'B01001_027E','B01001_028E', 'B01001_029E', 'B01001_030E', 'B01001_031E'])
df2 = df2.reset_index(drop=False)
df2.rename(columns = {'index' : 'TRACT', 'B01001_001E' : 'Total', 'B01001_002E' : 'Male', 'B01001_007E' : 'M 18-19', 'B01001_008E' : 'M 20-24', 
                             'B01001_009E' : 'M 25-29', 'B01001_010E' : 'M 30-34','B01001_011E' : 'M 35-44','B01001_012E' : 'M 45-54', 
                             'B01001_013E' : 'M 55-64', 'B01001_014E' : 'M 65-74', 'B01001_015E' : 'M 75-84', 
                             'B01001_016E' : 'M 85+', 'B01001_017E' : 'Female', 'B01001_022E' : 'F 18-19', 'B01001_023E' : 'F 20-24', 
                             'B01001_024E' : 'F 25-29', 'B01001_025E' : 'F 30-34', 'B01001_026E' : 'F 35-44', 'B01001_027E' : 'F 45-54',
                             'B01001_028E' : 'F 55-64', 'B01001_029E' : 'F 65-74', 'B01001_030E' : 'F 75-84', 'B01001_031E' : 'F 85+'}, 
                              inplace = True)
for x in range(len(df2)):
    df2.loc[x, 'TRACT'] = df2['TRACT'].unique()[x].geo[2][1]
    
df2 = df2.astype({"TRACT": int})

df2

,TRACT,Total,Male,M 18-19,M 20-24,M 25-29,M 30-34,M 35-44,M 45-54,M 55-64,...,F 18-19,F 20-24,F 25-29,F 30-34,F 35-44,F 45-54,F 55-64,F 65-74,F 75-84,F 85+
0,802,6701,3778,100,316,108,591,979,549,226,...,36,29,72,35,2923,41,39,97,103,124
1,10600,3168,1299,0,29,0,18,182,225,156,...,85,40,20,0,1869,43,12,0,0,34
2,40100,2442,1118,9,0,9,0,145,166,105,...,34,20,35,8,1324,82,27,0,8,0
3,40300,4665,2039,10,0,0,32,273,380,222,...,39,33,18,0,2626,364,233,56,0,21
4,50400,2284,1325,14,0,0,52,168,201,140,...,25,0,0,45,959,34,28,32,10,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,82100,5998,2659,55,49,38,29,360,355,84,...,72,74,144,81,3339,140,144,242,117,16
200,60200,2103,1014,0,5,5,42,267,184,60,...,23,15,6,12,1089,54,41,0,20,0
201,90400,4396,2013,127,112,32,75,178,63,127,...,28,12,52,0,2383,99,151,140,113,83
202,120500,2850,1449,17,6,0,94,301,233,105,...,14,38,0,25,1401,90,9,59,0,31


In [153]:
prct_age2 = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df2.columns[1:])

prct_age2.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_age2)):
    prct = prct_age2['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df2['TRACT'] == temp.loc[n, 'TRACT']
            for c in df2.columns[1:]:
                prct_age2[c].iloc[i] += round(df2[filt][c] * dec)

In [154]:
prct_age2.to_csv('Precinct_Age_Distribution_Based_On_2015-19_ACS_Census_Data.csv')

In [155]:
neigh_age_15_19 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_age2.columns[1:])
neigh_age_15_19.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_age_15_19)):
    neighb = neigh_age_15_19['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_age2['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_age2.columns[1:]:
                try:
                    neigh_age_15_19[c].iloc[i] += round(prct_age2[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_age_15_19.columns:
    if c != 'NEIGHBORHOOD':
        neigh_age_15_19[c] = neigh_age_15_19[c].round().astype(int)
        
sum_column = neigh_age_15_19['F 55-64'] + neigh_age_15_19['F 65-74'] + neigh_age_15_19['F 75-84'] + neigh_age_15_19['F 85+']
neigh_age_15_19.insert(23, 'F 55+', sum_column)

sum_column = neigh_age_15_19['M 55-64'] + neigh_age_15_19['M 65-74'] + neigh_age_15_19['M 75-84'] + neigh_age_15_19['M 85+']
neigh_age_15_19.insert(12, 'M 55+', sum_column)

sum_column = neigh_age_15_19['F 55+'] + neigh_age_15_19['M 55+']
neigh_age_15_19['Total 55+'] = sum_column

sum_column = neigh_age_15_19['F 18-19'] + neigh_age_15_19['F 20-24'] + neigh_age_15_19['F 25-29'] + \
             neigh_age_15_19['M 18-19'] + neigh_age_15_19['M 20-24'] + neigh_age_15_19['M 25-29']
neigh_age_15_19['Total 18-29'] = sum_column

cols = list(neigh_age_15_19.columns)[2:]

for c in cols:
    for i in range(len(neigh_age_15_19)):
        neigh_age_15_19.loc[i, c] = round(neigh_age_15_19.loc[i, c]/neigh_age_15_19.loc[i, 'Total'], 3)
        


neigh_age_15_19

,NEIGHBORHOOD,Total,Male,M 18-19,M 20-24,M 25-29,M 30-34,M 35-44,M 45-54,M 55-64,...,F 30-34,F 35-44,F 45-54,F 55-64,F 65-74,F 75-84,F 55+,F 85+,Total 55+,Total 18-29
0,Roslindale,30205,0.467,0.007,0.005,0.006,0.012,0.043,0.036,0.043,...,0.008,0.533,0.033,0.025,0.025,0.019,0.080,0.010,0.222,0.042
1,Jamaica Plain,43866,0.462,0.007,0.003,0.007,0.019,0.060,0.064,0.041,...,0.005,0.538,0.031,0.018,0.016,0.018,0.062,0.010,0.192,0.041
2,Mission Hill,14248,0.517,0.046,0.033,0.050,0.091,0.078,0.031,0.023,...,0.004,0.484,0.010,0.009,0.008,0.009,0.079,0.053,0.143,0.151
3,Longwood,6549,0.420,0.111,0.051,0.041,0.052,0.048,0.015,0.007,...,0.004,0.579,0.002,0.004,0.002,0.012,0.233,0.215,0.266,0.209
4,Bay Village,1465,0.472,0.027,0.009,0.003,0.016,0.041,0.072,0.027,...,0.011,0.528,0.026,0.007,0.007,0.008,0.052,0.031,0.174,0.079
5,Leather District,621,0.462,0.061,0.013,0.011,0.023,0.056,0.047,0.021,...,0.006,0.538,0.013,0.016,0.008,0.003,0.108,0.081,0.196,0.114
6,Chinatown,3113,0.463,0.058,0.013,0.010,0.022,0.056,0.049,0.022,...,0.007,0.536,0.014,0.016,0.008,0.004,0.103,0.075,0.194,0.111
7,North End,6784,0.505,0.003,0.003,0.009,0.041,0.152,0.090,0.034,...,0.002,0.495,0.009,0.011,0.007,0.000,0.022,0.004,0.104,0.049
8,Roxbury,53772,0.462,0.026,0.011,0.013,0.034,0.043,0.034,0.021,...,0.004,0.538,0.027,0.028,0.032,0.020,0.104,0.025,0.199,0.078
9,South End,24202,0.477,0.010,0.004,0.005,0.018,0.061,0.062,0.036,...,0.008,0.523,0.020,0.017,0.024,0.008,0.065,0.016,0.204,0.050


In [156]:
neigh_age_15_19.to_csv('Boston_Neighborhood_Estimated_Age_Distribution_ACS_2015-2019.csv')

In [150]:
#censusdata.printtable(censusdata.censustable('acs5', 2015, 'B01001'))

In [151]:
# H - White (Not Hispanic)
# B - African American -
# C - Native American
# D - Asian
# E - Native Hawaiian/Pacific Islander
# I - Hispanic/Latino

In [306]:
df = censusdata.download('acs5', 2019, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B01001H_001E', 'B01001H_002E', 'B01001H_007E', 'B01001H_008E', 'B01001H_009E', 'B01001H_010E',
                              'B01001H_011E','B01001H_012E', 'B01001H_013E', 'B01001H_014E', 'B01001H_015E', 'B01001H_016E', 
                              'B01001H_017E','B01001H_022E', 'B01001H_023E', 'B01001H_024E', 'B01001H_025E', 'B01001H_026E', 
                              'B01001H_027E','B01001H_028E', 'B01001H_029E', 'B01001H_030E', 'B01001H_031E'])
df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B01001H_001E' : 'Total', 'B01001H_002E' : 'Male', 'B01001H_007E' : 'M 18-19', 'B01001H_008E' : 'M 20-24', 
                             'B01001H_009E' : 'M 25-29', 'B01001H_010E' : 'M 30-34','B01001H_011E' : 'M 35-44','B01001H_012E' : 'M 45-54', 
                             'B01001H_013E' : 'M 55-64', 'B01001H_014E' : 'M 65-74', 'B01001H_015E' : 'M 75-84', 
                             'B01001H_016E' : 'M 85+', 'B01001H_017E' : 'Female', 'B01001H_022E' : 'F 18-19', 'B01001H_023E' : 'F 20-24', 
                             'B01001H_024E' : 'F 25-29', 'B01001H_025E' : 'F 30-34', 'B01001H_026E' : 'F 35-44', 'B01001H_027E' : 'F 45-54',
                             'B01001H_028E' : 'F 55-64', 'B01001H_029E' : 'F 65-74', 'B01001H_030E' : 'F 75-84', 'B01001H_031E' : 'F 85+'}, 
                              inplace = True)
for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

prct_age = pd.DataFrame(0, index=np.arange(len(template.columns[2:])),
                        columns=df.columns[1:])

prct_age.insert(0, 'WARD_PRECI', template.columns[2:])

for i in range(len(prct_age)):
    prct = prct_age['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            dec = temp.loc[n, prct]
            filt = df['TRACT'] == temp.loc[n, 'TRACT']
            for c in df.columns[1:]:
                prct_age[c].iloc[i] += round(df[filt][c] * dec)

prct_age.to_csv('Precinct_WHITE_Age_Distribution_Based_On_2015-19_ACS_Census_Data.csv')

neigh_age_11_15 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=prct_age.columns[1:])
neigh_age_11_15.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh_age_11_15)):
    neighb = neigh_age_11_15['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = prct_age['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            for c in prct_age.columns[1:]:
                try:
                    neigh_age_11_15[c].iloc[i] += round(prct_age[filt][c] * dec)
                except ValueError as v:
                    break

for c in neigh_age_11_15.columns:
    if c != 'NEIGHBORHOOD':
        neigh_age_11_15[c] = neigh_age_11_15[c].round().astype(int)

cols = list(neigh_age_11_15.columns)[2:]

#for c in cols:
    #for i in range(len(neigh_age_11_15)):
        #neigh_age_11_15.loc[i, c] = round(neigh_age_11_15.loc[i, c]/neigh_age_11_15.loc[i, 'Total'], 3)


In [307]:
neigh_age_11_15.to_csv('Boston_Neighborhood_Estimated_WHITE_Age_Distribution_ACS_2015-2019.csv')

In [308]:
total = pd.read_csv('Boston_Neighborhood_Estimated_Age_Distribution_ACS_2015-2019.csv')
#races = ['WHITE', 'AFRICAN_AMERICAN', 'NATIVE_AMERICAN', 'ASIAN']

new = pd.DataFrame(columns = ['Neighborhood', 'Total Pop.', '18-19', '20-24', '25-29', '30-34',
                              '35-44', '45-54', '55-64', '65-74', '75-84'])
title = 'Boston_Neighborhood_Estimated_WHITE_Age_Distribution_ACS_2015-2019.csv'
temp = pd.read_csv(title)
new['Total Pop.'] = total['Total']
new['Neighborhood'] = temp['NEIGHBORHOOD']
cols = temp.columns
cols = cols[4:13]
for i in range(len(cols)):
    fem = 'F ' + cols[i][2:]
    sum_column = temp[cols[i]] + temp[fem]
    new[cols[i][2:]] = sum_column

cols = list(new.columns)[2:]
for c in cols:
    for i in range(len(new)):
        new.loc[i, c] = round(new.loc[i, c]/new.loc[i, 'Total Pop.'], 3)
        

In [309]:
new.to_csv('Boston_Neighborhood_Estimated_WHITE_Age_Distribution_ACS_2015-2019.csv')

In [310]:
new

,Neighborhood,Total Pop.,18-19,20-24,25-29,30-34,35-44,45-54,55-64,65-74,75-84
0,Roslindale,30205,0.006,0.014,0.042,0.050,0.085,0.067,0.075,0.047,0.027
1,Jamaica Plain,43866,0.005,0.034,0.075,0.084,0.085,0.056,0.067,0.044,0.012
2,Mission Hill,14248,0.065,0.172,0.054,0.033,0.018,0.017,0.021,0.024,0.010
3,Longwood,6549,0.240,0.222,0.031,0.014,0.003,0.004,0.009,0.007,0.008
4,Bay Village,1465,0.040,0.057,0.074,0.111,0.057,0.076,0.072,0.046,0.033
5,Leather District,621,0.101,0.063,0.071,0.047,0.029,0.040,0.034,0.026,0.003
6,Chinatown,3113,0.096,0.062,0.071,0.052,0.031,0.043,0.039,0.027,0.006
7,North End,6784,0.005,0.089,0.267,0.140,0.095,0.049,0.069,0.061,0.034
8,Roxbury,53772,0.015,0.031,0.017,0.016,0.013,0.016,0.011,0.009,0.003
9,South End,24202,0.011,0.036,0.095,0.091,0.080,0.077,0.065,0.041,0.011
